In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#from torch.utils.data import DataLoader

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import os
import re
import random
from torch.utils.data.dataset import Dataset

In [11]:
open('enron1/ham/0310.2000-02-02.farmer.ham.txt', 'r').read()

'Subject: enron and chase manhattan bank sign long - term energy management\nagreement\nhouston and new york \x01 ) enron energy services , a subsidiary of enron corp . ,\nand the chase manhattan corporation ( nyse : cmb ) announced today a ten - year ,\n$ 750 million energy management agreement . enron will provide energy\nmanagement services for 860 of chase \x01 , s facilities nationwide , including\noffice buildings , retail branches and operations centers .\nas part of this agreement , enron will provide commodity management , including\nanalysis and consolidation of approximately 30 , 000 annual utility invoices\ncurrently processed through multiple operating groups , as well as overall\nproject management for energy infrastructure upgrades , designed to make\nmaterial improvements in the energy efficiency of many chase facilities .\n" chase has taken a leadership position among financial institutions by\nrecognizing the financial and administrative benefits of working with enron

In [3]:
def textParse(bigString):
    listOfTokens = re.split(r'\W+', bigString)  # 用重复任意次的非字符作为切分标志
    return [tok.lower() for tok in listOfTokens if len(tok) > 2]  # 将单词转化为小写

def createVocabList(docList):# docList：每一封邮件转化成的字符串列表，其中包含的是可能有重复的小写单词
    vocabSet = set([])# vocabSet：不重复的词汇表
    for document in docList:
        vocabSet = vocabSet | set(document)  #集合运算，通过取并集来保证不重复
    return list(vocabSet)

In [9]:
## 将数据以80%划为训练集和测试集
train_ratio = 0.8
train_list, test_list = [],[]
dataroot = r"data"
words = []
class_flag = -1
for a, b, f in os.walk(dataroot):
    print(a)
    for i in range(0, int(len(f)*train_ratio)):
        train_text = os.path.join(a, f[i]) +'\t'+str(class_flag)+'\n'
        train_list.append(train_text)
        words.append(textParse(open(os.path.join(a, f[i]), 'r',encoding='gbk',errors='ignore').read()))
    for i in range(int(len(f)*train_ratio),len(f)):
        test_text = os.path.join(a, f[i]) +'\t'+str(class_flag)+'\n'
        test_list.append(test_text)
        words.append(textParse(open(os.path.join(a, f[i]), 'r',encoding='gbk',errors='ignore').read()))
    class_flag += 1 
random.shuffle(train_list)
random.shuffle(test_list)

vocablist = createVocabList(words)
with open('train.txt', 'w') as f:
    for train_text in train_list:
        f.write(str(train_text))
with open('test.txt','w') as f:
    for test_text in test_list:
        f.write(str(test_text))    #0是ham， 1是spam

data
data\ham
data\spam


In [10]:
len(vocablist)
vocablist = sorted(vocablist)
i=0
while vocablist[i].isdigit():
    i=i+1
print(i) #从第4605开始 不再是数字 舍弃数字部分

4605


In [11]:
vocablist = vocablist[4605:]
len(vocablist)

45161

In [14]:
vocablist

['aaa',
 'aaas',
 'aabda',
 'aabvmmq',
 'aac',
 'aachecar',
 'aaer',
 'aafco',
 'aaiabe',
 'aaigrcrb',
 'aaihmqv',
 'aaldano',
 'aalland',
 'aambique',
 'aamlrg',
 'aaoeuro',
 'aare',
 'aarhus',
 'aaron',
 'aashqcsny',
 'aavilable',
 'aaxrzm',
 'aba',
 'ababa',
 'abacha',
 'aback',
 'abackof',
 'abacus',
 'abacustech',
 'abandon',
 'abandone',
 'abandoned',
 'abarch',
 'abasements',
 'abash',
 'abashed',
 'abate',
 'abater',
 'abazis',
 'abb',
 'abbas',
 'abbasi',
 'abbe',
 'abbey',
 'abbot',
 'abbott',
 'abbpge',
 'abbreviation',
 'abby',
 'abc',
 'abcdef',
 'abcdzhongguo',
 'abdomen',
 'abdominal',
 'abdominoplasty',
 'abduct',
 'abductbathe',
 'abduction',
 'abdul',
 'abdv',
 'abdvyyhtxnv',
 'abe',
 'abeckley',
 'abed',
 'abel',
 'abelian',
 'abelmosk',
 'abelson',
 'aber',
 'abercrombie',
 'aberdeen',
 'abernathy',
 'aberrant',
 'aberrate',
 'abet',
 'abetted',
 'abetting',
 'abeyant',
 'abfan',
 'abhorred',
 'abide',
 'abideth',
 'abidjan',
 'abie',
 'abigail',
 'abiiity',
 'abile

In [12]:
with open('vocablist.txt', 'w') as f:
    for word in vocablist:
        f.write(word+'\t')

In [4]:
vocab = list(open('vocablist.txt', 'r').read().strip().split('\t'))[1:]
len(vocab)

45160

In [5]:
class LoadData(Dataset):
    def __init__(self, txt_path, vocab):
        self.text_info = self.get_info(txt_path)
        self.vocab = vocab
        
    def get_info(self, txt_path):
        with open(txt_path, 'r') as f:
                text_info = f.readlines()
                text_info = list(map(lambda x:x.strip().split('\t'), text_info))
        return text_info


    # 将字符串转换为列表，0表示字符在词汇表中不存在，1表示字符在词汇表中存在
    def Words2Vec(self, inputSet):# vocabList：词汇表；inputSet：输入的字符串列表
        returnVec = [0] * 45160
        listOfTokens = re.split(r'\W+', inputSet)  # 用重复任意次的非字符作为切分标志
        inputSet = [tok.lower() for tok in listOfTokens if len(tok) > 2]
        for word in inputSet:
            if word in self.vocab:
                returnVec[self.vocab.index(word)] += 1
        return torch.Tensor(returnVec).reshape(1,1,45160)
    
    def __getitem__(self, index):
        email_path, label = self.text_info[index]
        email = open(email_path, 'r',encoding='gbk',errors='ignore').read()
        email = self.Words2Vec(email)
        label = int(label)
        return email, label
    
    def __len__(self):
        return len(self.text_info)

if __name__ == "__main__":
    train_data = LoadData("train.txt", vocab)
    print("训练集数据个数：",len(train_data))
    train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=128, shuffle=True, drop_last = True)
    test_data = LoadData("test.txt", vocab)
    print("测试集数据个数：",len(test_data))
    test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=128, shuffle=False)

训练集数据个数： 4137
测试集数据个数： 1035


In [ ]:
class Linear_BBB(nn.Module):
    """
        Layer of our BNN.
    """
    def __init__(self, input_features, output_features, prior_var=1.):
        """
            Initialization of our layer : our prior is a normal distribution
            centered in 0 and of variance 20.
        """
        # initialize layers
        super().__init__()
        # set input and output dimensions
        self.input_features = input_features
        self.output_features = output_features

        # initialize mu and rho parameters for the weights of the layer
        self.w_mu = nn.Parameter(torch.zeros(output_features, input_features))
        self.w_rho = nn.Parameter(torch.zeros(output_features, input_features))

        #initialize mu and rho parameters for the layer's bias
        self.b_mu =  nn.Parameter(torch.zeros(output_features))
        self.b_rho = nn.Parameter(torch.zeros(output_features))        

        #initialize weight samples (these will be calculated whenever the layer makes a prediction)
        self.w = None
        self.b = None

        # initialize prior distribution for all of the weights and biases
        self.prior = Normal(0,prior_var)

    def forward(self, input):
        """
          Optimization process
        """
        # sample weights
        w_epsilon = Normal(0,1).sample(self.w_mu.shape)
        self.w = self.w_mu + torch.log(1+torch.exp(self.w_rho)) * w_epsilon

        # sample bias
        b_epsilon = Normal(0,1).sample(self.b_mu.shape)
        self.b = self.b_mu + torch.log(1+torch.exp(self.b_rho)) * b_epsilon

        # record log prior by evaluating log pdf of prior at sampled weight and bias
        w_log_prior = self.prior.log_prob(self.w)
        b_log_prior = self.prior.log_prob(self.b)
        self.log_prior = torch.sum(w_log_prior) + torch.sum(b_log_prior)

        # record log variational posterior by evaluating log pdf of normal distribution defined by parameters with respect at the sampled values
        self.w_post = Normal(self.w_mu.data, torch.log(1+torch.exp(self.w_rho)))
        self.b_post = Normal(self.b_mu.data, torch.log(1+torch.exp(self.b_rho)))
        self.log_post = self.w_post.log_prob(self.w).sum() + self.b_post.log_prob(self.b).sum()

        return F.linear(input, self.w, self.b)

class MLP_BBB(nn.Module):
    def __init__(self, hidden_units, noise_tol=.1,  prior_var=1.):

        # initialize the network like you would with a standard multilayer perceptron, but using the BBB layer
        super().__init__()
        self.hidden = Linear_BBB(45160,hidden_units, prior_var=prior_var)
        self.out = Linear_BBB(hidden_units, 45160, prior_var=prior_var)
        self.noise_tol = noise_tol # we will use the noise tolerance to calculate our likelihood

    def forward(self, x):
        # again, this is equivalent to a standard multilayer perceptron
        x = torch.sigmoid(self.hidden(x))
        x = self.out(x)
        return x

    def log_prior(self):
        # calculate the log prior over all the layers
        return self.hidden.log_prior + self.out.log_prior

    def log_post(self):
        # calculate the log posterior over all the layers
        return self.hidden.log_post + self.out.log_post

    def sample_elbo(self, input, target, samples):
        # we calculate the negative elbo, which will be our loss function
        #initialize tensors
        outputs = torch.zeros(samples, target.shape[0])
        log_priors = torch.zeros(samples)
        log_posts = torch.zeros(samples)
        log_likes = torch.zeros(samples)
        # make predictions and calculate prior, posterior, and likelihood for a given number of samples
        for i in range(samples):
            outputs[i] = self(input).reshape(-1) # make predictions
            log_priors[i] = self.log_prior() # get log prior
            log_posts[i] = self.log_post() # get log variational posterior
            log_likes[i] = Normal(outputs[i], self.noise_tol).log_prob(target.reshape(-1)).sum() # calculate the log likelihood
        # calculate monte carlo estimate of prior posterior and likelihood
        log_prior = log_priors.mean()
        log_post = log_posts.mean()
        log_like = log_likes.mean()
        # calculate the negative elbo (which is our loss function)
        loss = log_post - log_prior - log_like
        return loss

torch.cuda.get_device_name(torch.cuda.current_device())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# x = torch.unsqueeze(torch.linspace(-1,1,100),dim=1)
# y = x.pow(3)+0.1*torch.randn(x.size())

net = MLP_BBB(48, prior_var=10)
optimizer = optim.Adam(net.parameters(), lr=.1)
epochs = 5000
hist_epochs = np.zeros((int(epochs/10),1))
hist_loss = np.zeros((int(epochs/10),1))
for epoch in range(epochs):  # loop over the dataset multiple times
    optimizer.zero_grad()
    # forward + backward + optimize
    for batch_id, data in enumerate(train_loader):
        # calculate the loss and take a gradient step
        loss += svi.step(data[0].view(-1,45160), data[1])
        loss = net.sample_elbo(x, y, 1)
      loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        hist_loss[int(epoch/10)] = loss.data
        hist_epochs[int(epoch/10)] = epoch+1
        print('epoch: {}/{}'.format(epoch+1,epochs))
        print('Loss: %.4f' % loss.item())
print('Finished Training')

In [ ]:
# samples is the number of "predictions" we make for 1 x-value.
samples = 500
x_tmp = torch.linspace(-2,2,100).reshape(-1,1)
y_samp = np.zeros((samples,100))
for s in range(samples):
    y_tmp = net(x_tmp).detach().numpy()
    y_samp[s] = y_tmp.reshape(-1)


y_sigma = np.std(y_samp, axis=0)
y_mean = np.mean(y_samp, axis = 0)

plt.figure(1)
plt.plot(x_tmp.numpy(), y_mean, label='Mean Posterior Predictive')
plt.fill_between(x_tmp.numpy().reshape(-1), y_mean + 2 * y_sigma,  y_mean - 2 * y_sigma, alpha=0.25, label='Epistemic uncertainty')
plt.legend()
plt.scatter(x, y)
plt.title('Posterior Predictive')

plt.figure(2)
plt.plot(hist_epochs,hist_loss, label='Loss Function')
plt.legend()
plt.title('The Value of loss')
plt.show()

In [6]:
class NN(nn.Module):
    
    def __init__(self, input_size, hidden_size, output_size):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        output = self.fc1(x)
        output = F.relu(output)
        output = self.out(output)
        return output

In [7]:
net = NN(45160, 512, 2)

In [8]:
import pyro
from pyro.distributions import Normal, Categorical
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam

In [9]:
log_softmax = nn.LogSoftmax(dim=1)

In [10]:
def model(x_data, y_data):
    
    fc1w_prior = Normal(loc=torch.zeros_like(net.fc1.weight), scale=torch.ones_like(net.fc1.weight))
    fc1b_prior = Normal(loc=torch.zeros_like(net.fc1.bias), scale=torch.ones_like(net.fc1.bias))
    
    outw_prior = Normal(loc=torch.zeros_like(net.out.weight), scale=torch.ones_like(net.out.weight))
    outb_prior = Normal(loc=torch.zeros_like(net.out.bias), scale=torch.ones_like(net.out.bias))
    
    priors = {'fc1.weight': fc1w_prior, 'fc1.bias': fc1b_prior,  'out.weight': outw_prior, 'out.bias': outb_prior}
    # lift module parameters to random variables sampled from the priors
    lifted_module = pyro.random_module("module", net, priors)
    # sample a regressor (which also samples w and b)
    lifted_reg_model = lifted_module()
    
    lhat = log_softmax(lifted_reg_model(x_data))
    
    pyro.sample("obs", Categorical(logits=lhat), obs=y_data)

In [11]:
softplus = torch.nn.Softplus()
def guide(x_data, y_data):
    
    # First layer weight distribution priors
    fc1w_mu = torch.randn_like(net.fc1.weight)
    fc1w_sigma = torch.randn_like(net.fc1.weight)
    fc1w_mu_param = pyro.param("fc1w_mu", fc1w_mu)
    fc1w_sigma_param = softplus(pyro.param("fc1w_sigma", fc1w_sigma))
    fc1w_prior = Normal(loc=fc1w_mu_param, scale=fc1w_sigma_param)
    # First layer bias distribution priors
    fc1b_mu = torch.randn_like(net.fc1.bias)
    fc1b_sigma = torch.randn_like(net.fc1.bias)
    fc1b_mu_param = pyro.param("fc1b_mu", fc1b_mu)
    fc1b_sigma_param = softplus(pyro.param("fc1b_sigma", fc1b_sigma))
    fc1b_prior = Normal(loc=fc1b_mu_param, scale=fc1b_sigma_param)
    # Output layer weight distribution priors
    outw_mu = torch.randn_like(net.out.weight)
    outw_sigma = torch.randn_like(net.out.weight)
    outw_mu_param = pyro.param("outw_mu", outw_mu)
    outw_sigma_param = softplus(pyro.param("outw_sigma", outw_sigma))
    outw_prior = Normal(loc=outw_mu_param, scale=outw_sigma_param).independent(1)
    # Output layer bias distribution priors
    outb_mu = torch.randn_like(net.out.bias)
    outb_sigma = torch.randn_like(net.out.bias)
    outb_mu_param = pyro.param("outb_mu", outb_mu)
    outb_sigma_param = softplus(pyro.param("outb_sigma", outb_sigma))
    outb_prior = Normal(loc=outb_mu_param, scale=outb_sigma_param)
    priors = {'fc1.weight': fc1w_prior, 'fc1.bias': fc1b_prior, 'out.weight': outw_prior, 'out.bias': outb_prior}
    
    lifted_module = pyro.random_module("module", net, priors)
    
    return lifted_module()

In [12]:
optim = Adam({"lr": 0.01})
svi = SVI(model, guide, optim, loss=Trace_ELBO())

In [63]:
for data ,label in train_loader:
    print(data.shape)
    print(label)
    break

torch.Size([128, 1, 1129, 40])
tensor([0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,
        0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,
        0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1])


In [13]:
num_iterations = 5
loss = 0

for j in range(num_iterations):
    loss = 0
    for batch_id, data in enumerate(train_loader):
        # calculate the loss and take a gradient step
        loss += svi.step(data[0].view(-1,45160), data[1])
    normalizer_train = len(train_loader.dataset)
    total_epoch_loss_train = loss / normalizer_train
    
    print("Epoch ", j, " Loss ", total_epoch_loss_train)

KeyboardInterrupt: 

In [12]:
num_samples = 2
def predict(x):
    sampled_models = [guide(None, None) for _ in range(num_samples)]
    yhats = [model(x).data for model in sampled_models]
    mean = torch.mean(torch.stack(yhats), 0)
    return np.argmax(mean.numpy(), axis=1)

print('Prediction when network is forced to predict')
correct = 0
total = 0
for j, data in enumerate(test_loader):
    images, labels = data
    predicted = predict(images.view(-1,1129*40))
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
print("accuracy: %d %%" % (100 * correct / total))

Prediction when network is forced to predict


AttributeError: 'bool' object has no attribute 'sum'

In [ ]:
# Prediction when network can decide not to predict

print('Prediction when network can refuse')
correct = 0
total = 0
total_predicted_for = 0
for j, data in enumerate(test_loader):
    images, labels = data
    
    total_minibatch, correct_minibatch, predictions_minibatch = test_batch(images, labels, plot=False)
    total += total_minibatch
    correct += correct_minibatch
    total_predicted_for += predictions_minibatch

print("Total images: ", total)
print("Skipped: ", total-total_predicted_for)
print("Accuracy when made predictions: %d %%" % (100 * correct / total_predicted_for))

In [18]:
classes = ('0', '1', '2', '3',
           '4', '5', '6', '7', '8', '9')

In [20]:
num_samples = 100
def give_uncertainities(x):
    sampled_models = [guide(None, None) for _ in range(num_samples)]
    yhats = [F.log_softmax(model(x.view(-1,28*28)).data, 1).detach().numpy() for model in sampled_models]
    return np.asarray(yhats)
    #mean = torch.mean(torch.stack(yhats), 0)
    #return np.argmax(mean, axis=1)